In [1]:

####################
#Author: brandon chiazza
#version: 1.0
#purpose: to call a twitter api and return results
#documentation: https://developer.twitter.com/en/docs
#####################

import pandas as pd
import requests
import json
import base64
import s3fs # documentation: https://s3fs.readthedocs.io/en/latest/
import time
import twitter_keys #this is a custom reference module to a package containing twitter keys

%config IPCompleter.greedy=True


key_secret = '{}:{}'.format(twitter_keys.client_key, twitter_keys.client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

#identify base url and oauth token path
base_url = 'https://api.twitter.com/' #base url for authentication
auth_url = '{}oauth2/token'.format(base_url)

#share header information -- encoding is ascii
auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

#pass clientcredentials
auth_data = {
    'grant_type': 'client_credentials'
}

#send authentication using requests - POST request
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

#check response status. 200 = OK
auth_resp.status_code




200

In [2]:

#Keys in data response are token_type (bearer) and access_token (your access token)
print(auth_resp.json().keys())

access_token = auth_resp.json()['access_token']


search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

#enter search parameters for coronavirus example. This looks for "covid-19" in the 1000 most recent tweets
query_params = {
    'q': 'Olympic',
    'result_type': 'recent',
    'count': 1000, #update here to get more/less than 1000 returns
    'lang': 'en' #filters by english language only
}


#identify search url path and save 
search_url = '{}1.1/search/tweets.json'.format(base_url)


#run search using get request
search_resp = requests.get(search_url, headers=search_headers, params=query_params)

#check status code of GET request
search_resp.status_code


dict_keys(['token_type', 'access_token'])


200

In [3]:
#print text from result to verify  
twitter_data = search_resp.json()

for x in twitter_data['statuses']:
    print(x['text'] + '\n')
    break #prints after one iteration and stops, remove break to see all 1000

RT @touringdata: .@BTS_twt's #PTD_ON_STAGE_SEOUL earned a total of $62.5 million from 1.065 million tickets sold at Olympic Stadium in Seou…



In [4]:
# move data into data frame 
df = pd.DataFrame(twitter_data['statuses'])

# show one record to verify import 
df.head(20)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,Sat Apr 02 20:18:19 +0000 2022,1510350939877748742,1510350939877748742,RT @touringdata: .@BTS_twt's #PTD_ON_STAGE_SEO...,False,"{'hashtags': [{'text': 'PTD_ON_STAGE_SEOUL', '...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,6823,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Sat Apr 02 20:18:12 +0000 2022,1510350908475199488,1510350908475199488,RT @Hoophall: Congratulations to 4x @NBA Champ...,False,"{'hashtags': [{'text': '22HoopClass', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,2050,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Sat Apr 02 20:17:56 +0000 2022,1510350840892076035,1510350840892076035,https://t.co/aCLALOUBiD Adam Peaty on staying ...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,0,False,False,en,"{'media': [{'id': 1430726403121577986, 'id_str...",False,NaN,NaN,NaN
3,Sat Apr 02 20:17:54 +0000 2022,1510350833711403008,1510350833711403008,I mean I’m sure she got paid for the Olympic g...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,0,False,False,en,NaN,False,1.510044e+18,1510044394258239488,{'created_at': 'Sat Apr 02 00:00:13 +0000 2022...
4,Sat Apr 02 20:17:54 +0000 2022,1510350832524476420,1510350832524476420,RT @Hoophall: Congratulations to 4x @NBA Champ...,False,"{'hashtags': [{'text': '22HoopClass', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,2050,0,False,False,en,NaN,NaN,NaN,NaN,NaN
5,Sat Apr 02 20:17:52 +0000 2022,1510350826753040389,1510350826753040389,"RT @mbk_center: In Penza🇷🇺, students of the Ol...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,534,0,False,False,en,NaN,NaN,NaN,NaN,NaN
6,Sat Apr 02 20:17:37 +0000 2022,1510350761883889664,1510350761883889664,RT @userbfIy: WAIT. SOME OF YALL HAVE NEVER EX...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,2520,0,False,False,en,NaN,NaN,NaN,NaN,NaN
7,Sat Apr 02 20:17:33 +0000 2022,1510350744494477319,1510350744494477319,"RT @touringdata: PERMISSION TO DANCE ON STAGE,...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,3612,0,False,False,en,NaN,NaN,NaN,NaN,NaN
8,Sat Apr 02 20:17:30 +0000 2022,1510350732293292035,1510350732293292035,RT @Hoophall: Recognized posthumously is Radiv...,False,"{'hashtags': [{'text': '22HoopClass', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,79,0,False,False,en,NaN,NaN,NaN,NaN,NaN
9,Sat Apr 02 20:17:24 +0000 2022,1510350706385039373,1510350706385039373,RT @Hoophall: Congratulations to 3x @WNBA Cham...,False,"{'hashtags': [{'text': '22HoopClass', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,179,0,False,False,en,NaN,NaN,NaN,NaN,NaN


In [5]:
# we can use pandas to put data directly into an s3 bucket
#prepare csv file name   
filename = 'lab-03/'#specify location of s3:/{my-bucket}/
groupname= 'Group_teli_' #name of your group
datetime = time.strftime("%Y%m%d") #timestamp
filenames3 = "%s%s%s.csv"%(filename,groupname,datetime) #name of the filepath and csv file
print(filenames3)


#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df.to_csv(None).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames3, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

lab-03/Group_teli_20220402.csv
Successfull uploaded file to location:lab-03/Group_teli_20220402.csv
